Based on https://huggingface.co/state-spaces/mamba-130m-hf, but forcing it to use GPU only to prevent issues.

I outline 2 bugfixes that I needed: for Mamba and Triton.

Learning Plug: I'm learning modern AI / machine learning. If you want someone to mentor, please teach me something.

In [15]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Thu Mar  7 00:58:15 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:41:00.0  On |                  Off |
| 32%   42C    P8    31W / 450W |   8859MiB / 24564MiB |     43%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
%pip install -q wheel setuptools pip --upgrade
%pip install -q causal-conv1d
%pip install -q mamba-ssm
%pip install -q git+https://github.com/huggingface/transformers@main

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


## Generation with Base Model

In [17]:
from transformers import MambaConfig, MambaForCausalLM, AutoTokenizer

In [18]:
device = 'cuda'

In [19]:
model = MambaForCausalLM.from_pretrained("state-spaces/mamba-130m-hf")
model.to(device)

tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-130m-hf", device=device)

input_ids = tokenizer("What is Universal Basic Income (UBI)?", return_tensors="pt")["input_ids"]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [20]:
# RuntimeError: Cannot find ptxas
# There is a bug in Triton's backend.py
"""
def _path_to_binary(binary: str):
    base_dir = os.path.join(os.path.dirname(__file__), os.pardir)
    paths = [
        os.environ.get(f"TRITON_{binary.upper()}_PATH", ""),
        os.path.join(base_dir, "third_party", "cuda", "bin", binary)
    ]

    for p in paths:
        bin = p.split(" ")[0] # Bug! If there are spaces in the path to your ipynb, you will have a bad time.
        bin = p
"""

out = model.generate(input_ids.to(device), max_new_tokens=10)  # RuntimeError: Cannot find ptxas

print(tokenizer.batch_decode(out))

['What is Universal Basic Income (UBI)?\n\nUniversal Basic Income (UBI) is']


## Finetuning

In [21]:
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-130m-hf")

model = AutoModelForCausalLM.from_pretrained("state-spaces/mamba-130m-hf")
model.to(device)

dataset = load_dataset("Abirate/english_quotes", split="train")
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,  # Was 3.
    per_device_train_batch_size=4,
    logging_dir='./logs',
    logging_steps=10,
    learning_rate=2e-3
)
lora_config =  LoraConfig(
        r=8,
        target_modules=["x_proj", "embeddings", "in_proj", "out_proj"],
        task_type="CAUSAL_LM",
        bias="none"
)
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    peft_config=lora_config,
    train_dataset=dataset,
    dataset_text_field="quote",
)

trainer.train()

# {'train_runtime': 157.0453, 'train_samples_per_second': 47.91, 'train_steps_per_second': 11.977, 'train_loss': 2.8722433143952375, 'epoch': 3.0}

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/mnt/Hard_Drive_A/repos/ai_nlp/squire/.venv_squire_finetuning/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:225: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
  2%|▏         | 12/627 [00:00<00:47, 12.94it/s]

{'loss': 3.5187, 'grad_norm': 1.9333045482635498, 'learning_rate': 0.001968102073365231, 'epoch': 0.02}


  4%|▎         | 22/627 [00:01<00:45, 13.22it/s]

{'loss': 3.1839, 'grad_norm': 1.6528894901275635, 'learning_rate': 0.0019362041467304626, 'epoch': 0.03}


  5%|▌         | 32/627 [00:02<00:43, 13.61it/s]

...

 99%|█████████▉| 622/627 [00:45<00:00, 12.60it/s]

{'loss': 3.2488, 'grad_norm': 0.6396878957748413, 'learning_rate': 2.2328548644338118e-05, 'epoch': 0.99}


100%|██████████| 627/627 [00:46<00:00, 13.57it/s]

{'train_runtime': 46.2065, 'train_samples_per_second': 54.278, 'train_steps_per_second': 13.57, 'train_loss': 3.2073059629595453, 'epoch': 1.0}


TrainOutput(global_step=627, training_loss=3.2073059629595453, metrics={'train_runtime': 46.2065, 'train_samples_per_second': 54.278, 'train_steps_per_second': 13.57, 'train_loss': 3.2073059629595453, 'epoch': 1.0})

In [22]:
model.to(device)

MambaForCausalLM(
  (backbone): MambaModel(
    (embeddings): lora.Embedding(
      (base_layer): Embedding(50280, 768)
      (lora_dropout): ModuleDict(
        (default): Identity()
      )
      (lora_A): ModuleDict()
      (lora_B): ModuleDict()
      (lora_embedding_A): ParameterDict(  (default): Parameter containing: [torch.cuda.FloatTensor of size 8x50280 (cuda:0)])
      (lora_embedding_B): ParameterDict(  (default): Parameter containing: [torch.cuda.FloatTensor of size 768x8 (cuda:0)])
    )
    (layers): ModuleList(
      (0-23): 24 x MambaBlock(
        (norm): MambaRMSNorm()
        (mixer): MambaMixer(
          (conv1d): Conv1d(1536, 1536, kernel_size=(4,), stride=(1,), padding=(3,), groups=1536)
          (act): SiLU()
          (in_proj): lora.Linear(
            (base_layer): Linear(in_features=768, out_features=3072, bias=False)
            (lora_dropout): ModuleDict(
              (default): Identity()
            )
            (lora_A): ModuleDict(
              (de

In [23]:
input_ids = tokenizer("What is Universal Basic Income (UBI)?", return_tensors="pt")["input_ids"].to(device)
print(input_ids)

tensor([[ 1276,   310, 21312, 20233, 33224,   313,    54,  6159,  6177]],
       device='cuda:0')


In [24]:
# Bug fix in Mamba modeling_mamba.py

"""

    def _update_model_kwargs_for_generation(
        self, outputs: ModelOutput, model_kwargs: Dict[str, Any], **kwargs
    ) -> Dict[str, Any]:

        if "cache_params" in outputs:  # Add this line for safety.
            model_kwargs["cache_params"] = outputs["cache_params"]
        return model_kwargs

"""

# I also got this error before I used .to(device) everywhere.
# RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

out = model.generate(input_ids, max_new_tokens=10)  

In [25]:
input_text = "What is Universal Basic Income (UBI)?"
input_ids = tokenizer(input_text, return_tensors="pt")["input_ids"]

input_ids = input_ids.to(model.device)

# Generate text
generated = model.generate(input_ids, max_length=50, temperature=0.7)

# Decode and print generated text
decoded_sequence = tokenizer.decode(generated[0], skip_special_tokens=True)
print(decoded_sequence)

/mnt/Hard_Drive_A/repos/ai_nlp/squire/.venv_squire_finetuning/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


What is Universal Basic Income (UBI)? It is a program of universal basic income (UBI) that is based on the idea that the basic income is a universal basic income. Universal basic income is a program of universal basic income (UBI
